In [1]:
import pymongo 
import json 
from tqdm import tqdm

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.visualizing_sep

In [1]:
def create_sep_network_json(sep_collection):
    """ exports the JSON that makes up the network graph 

    Things to watch for in the export file: 
    1. entries with 'entries//'
    2. depiction needs a fix at 'reference'
    3. type theory needs a fix for 'russell'
    """
    #init empty lists 
    nodes_list = []
    links_list = []

    sep_entries = list(sep_collection.find({}).sort('title'))

    #loop through all entries
    for entry in tqdm(sep_entries, desc='Processing'):
        inpho_data = entry['inpho_api']
        
        if 'thinker' in inpho_data:
            entry_type = 'thinker'
        else:
            entry_type = 'idea'

        node_object = { 
            'id': entry['page_url'], 
            'title': entry['title'],
            'entry_type': entry_type,
            'first_paragraph': entry["first_paragraph"]
            }

        nodes_list.append(node_object)

        for link in entry['outlinks']:
            links_list.append({'source':entry['page_url'], 'target':link['link']})

    network_object = {'nodes': nodes_list, 'links':sorted(links_list)}

    with open('network1.json', 'w', encoding='UTF-8') as f:
        json.dump(network_object,f,ensure_ascii=False, indent=4)

In [5]:
#get sep entries stored in mongo
sep_entries = db.sep_data
create_sep_network_json(sep_entries)



Processing: 100%|██████████| 1680/1680 [00:00<00:00, 240615.70it/s]
